In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '/kaggle/input/mobile-games-ab-testing/cookie_cats.csv'

df = pd.read_csv(path)
df.head(5)

In [ ]:
df.shape

In [ ]:
len(set(df['userid'])) == df.shape[0]

# it seem that there is no repeatable value for user

In [ ]:
df.describe([0.01,0.05, 0.10, 0.20, 0.80, 0.90, 0.95, 0.99])[['sum_gamerounds']].T

In [ ]:
df.groupby('version')['sum_gamerounds'].agg(['count','median','mean','std','max'])

In [ ]:
#the max of gate_30 is 49854 gamerounds however if we check 99% confidence interval for sum_gamerounds which is 493


In [ ]:
df = df[df['sum_gamerounds'] < df['sum_gamerounds'].max()]

In [ ]:
df[df['sum_gamerounds'] == 0 ].shape[0]
# 3994 users did not try to play our game app

In [ ]:
df['sum_gamerounds'].value_counts()

In [ ]:

sns.lineplot(x=df['sum_gamerounds'].value_counts().index, y = df['sum_gamerounds'].value_counts().values)
plt.show()

In [ ]:
df.groupby('sum_gamerounds')['userid'].nunique().reset_index().head(10)

In [ ]:
# A/B Groups & Target Summary Stats
df.groupby("version").sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

In [ ]:
# our control and test groups have approximately same sample 

In [ ]:
ret1_count = df['retention_1'].sum()
ret7_count = df['retention_7'].sum()


In [ ]:
df['retention_1'].mean()


In [ ]:
df['retention_7'].mean()
#18% percentage people keep continue to play game

In [ ]:
df.groupby(["version", "retention_1"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

In [ ]:
df.groupby(["version", "retention_7"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

In [ ]:
# we checked that  have the control and test group same quantity for retention_1 and retention_7 ? YES!

In [ ]:
# Define A/B groups
df["version"] = np.where(df.version == "gate_30", "A", "B")
df.head()

In [ ]:
# A/B Testing Function - Quick Solution
def AB_Test(dataframe, group, target):
    
    # Packages
    from scipy.stats import shapiro
    import scipy.stats as stats
    
    # Split A/B
    groupA = dataframe[dataframe[group] == "A"][target]
    groupB = dataframe[dataframe[group] == "B"][target]
    
    # Assumption: Normality
    ntA = shapiro(groupA)[1] < 0.05
    ntB = shapiro(groupB)[1] < 0.05
    # H0: Distribution is Normal! - False
    # H1: Distribution is not Normal! - True
    
    if (ntA == False) & (ntB == False): # "H0: Normal Distribution"
        # Parametric Test
        # Assumption: Homogeneity of variances
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        # H0: Homogeneity: False
        # H1: Heterogeneous: True
        
        if leveneTest == False:
            # Homogeneity
            ttest = stats.ttest_ind(groupA, groupB, equal_var=True)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Heterogeneous
            ttest = stats.ttest_ind(groupA, groupB, equal_var=False)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Non-Parametric Test
        ttest = stats.mannwhitneyu(groupA, groupB)[1] 
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True
        
    # Result
    temp = pd.DataFrame({
        "AB Hypothesis":[ttest < 0.05], 
        "p-value":[ttest]
    })
    temp["Test Type"] = np.where((ntA == False) & (ntB == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A/B groups are similar!", "A/B groups are not similar!")
    
    # Columns
    if (ntA == False) & (ntB == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Test Type", "Homogeneity","AB Hypothesis", "p-value", "Comment"]]
    else:
        temp = temp[["Test Type","AB Hypothesis", "p-value", "Comment"]]
    
    # Print Hypothesis
    print("# A/B Testing Hypothesis")
    print("H0: A == B")
    print("H1: A != B", "\n")
    
    return temp
    
    
    
# Apply A/B Testing
AB_Test(dataframe=df, group = "version", target = "sum_gamerounds")

In [ ]:
# the A/B test function recejt the our H0 hypothesis so the groups are not similar to make a interpret.


df.groupby('version')['retention_1'].mean()

In [ ]:
df.groupby('version')['retention_7'].mean()

In [ ]:
# Group A which is gate_30 has more average ratio than group b for sum_gamerounds